In [1]:
import os, glob, random, soundfile, torch, sys, tqdm, time, pickle, numpy, argparse, warnings, math, librosa
import numpy as np
import pandas as pd

from sklearn import metrics
from operator import itemgetter

import torchvision.models as models
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torchaudio
from torch import flatten
from scipy import signal

# 在官方的base的基础上，增加上1st的模型
# 再此基础上，修改loss 和 optimi
# 增加上drop_aug
- 改进
    - 1.对每个batch，都进行转图谱 + 增强操作（生成双倍的数据）（先放弃）
    - 2.使用1st的模型

## 官方base的功能函数

In [2]:
def init_args(args):
    args.score_save_path    = os.path.join(args.save_path, 'score.txt')
    args.model_save_path    = os.path.join(args.save_path, 'base_1st_res_model')
    os.makedirs(args.model_save_path, exist_ok = True)
    return args

def metrics_scores(output, target):
    # output = output.detach().cpu().numpy().argmax(axis=1)
    output = output.detach().cpu().numpy()
    output = output >= 0.5  # 这会创建一个布尔数组，True 表示原值大于0.5，False 表示小于等于0.5
    output = output.astype(int)  # 将布尔值转换为整数，True 变为 1，False 变为 0
    
    target = target.detach().cpu().numpy()
    accuracy = metrics.accuracy_score(target, output)
    recall = metrics.recall_score(target, output)
    precision = metrics.precision_score(target, output)
    F1 = metrics.f1_score(target, output)   
    return accuracy*100, recall*100, precision, F1


class train_loader(object):
    def __init__(self, train_list, num_frames, **kwargs):
        self.num_frames = num_frames
        # Load data & labels
        df_train = pd.read_csv(train_list)
        self.data_list = df_train['wav_path'].tolist()
        self.data_label = df_train['label'].tolist()

    def __getitem__(self, index):
        # Read the utterance and randomly select the segment
        # 每次获取数据时随机选择一个片段进行训练，以增加数据多样性。
        audio, sr = soundfile.read(self.data_list[index])
        length = self.num_frames * 80
        if audio.shape[0] <= length:
            shortage = length - audio.shape[0]
            audio = numpy.pad(audio, (0, shortage), 'wrap')
        start_frame = numpy.int64(random.random()*(audio.shape[0]-length))
        audio = audio[start_frame:start_frame + length]
        audio = numpy.stack([audio],axis=0)  # [1, 1600]
        return torch.FloatTensor(audio[0]), self.data_label[index]

    def __len__(self):
        return len(self.data_list)

## 1st的功能函数
- 1.对频谱图求log
- 2.对频谱图做随机drop

In [3]:
# 生成离散傅里叶变换（DFT）和逆离散傅里叶变换（IDFT）矩阵。
class DFTBase(nn.Module):
    def __init__(self):
        """Base class for DFT and IDFT matrix"""
        super(DFTBase, self).__init__()

    def dft_matrix(self, n):
        (x, y) = np.meshgrid(np.arange(n), np.arange(n))
        omega = np.exp(-2 * np.pi * 1j / n)
        W = np.power(omega, x * y)
        return W

    def idft_matrix(self, n):
        (x, y) = np.meshgrid(np.arange(n), np.arange(n))
        omega = np.exp(2 * np.pi * 1j / n)
        W = np.power(omega, x * y)
        return W
    
# 继承自 DFTBase，实现了使用卷积1D进行短时傅里叶变换（STFT）。
class STFT(DFTBase):
    def __init__(self, n_fft=2048, hop_length=None, win_length=None, window='hann', 
                 center=True, pad_mode='reflect', freeze_parameters=True):
        
        """Implementation of STFT with Conv1d. The function has the same output 
        of librosa.core.stft
        """
        # 使用 super()，调用父类（超类）的构造函数
        super(STFT, self).__init__()

        assert pad_mode in ['constant', 'reflect']

        self.n_fft = n_fft
        self.center = center
        self.pad_mode = pad_mode

        # By default, use the entire frame
        if win_length is None:
            win_length = n_fft

        # Set the default hop, if it's not already specified
        if hop_length is None:
            hop_length = int(win_length // 4)

        fft_window = librosa.filters.get_window(window, win_length, fftbins=True)

        # Pad the window out to n_fft size
        fft_window = librosa.util.pad_center(fft_window, size = n_fft)

        # DFT & IDFT matrix
        self.W = self.dft_matrix(n_fft)

        out_channels = n_fft // 2 + 1

        self.conv_real = nn.Conv1d(in_channels=1, out_channels=out_channels, 
            kernel_size=n_fft, stride=hop_length, padding=0, dilation=1, 
            groups=1, bias=False)

        self.conv_imag = nn.Conv1d(in_channels=1, out_channels=out_channels, 
            kernel_size=n_fft, stride=hop_length, padding=0, dilation=1, 
            groups=1, bias=False)

        self.conv_real.weight.data = torch.Tensor(
            np.real(self.W[:, 0 : out_channels] * fft_window[:, None]).T)[:, None, :]
        # (n_fft // 2 + 1, 1, n_fft)

        self.conv_imag.weight.data = torch.Tensor(
            np.imag(self.W[:, 0 : out_channels] * fft_window[:, None]).T)[:, None, :]
        # (n_fft // 2 + 1, 1, n_fft)

        if freeze_parameters:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, input):
        """input: (batch_size, data_length)
        Returns:
          real: (batch_size, n_fft // 2 + 1, time_steps)
          imag: (batch_size, n_fft // 2 + 1, time_steps)
        """

        x = input[:, None, :]   # (batch_size, channels_num, data_length)

        if self.center:
            x = F.pad(x, pad=(self.n_fft // 2, self.n_fft // 2), mode=self.pad_mode)

        real = self.conv_real(x)
        imag = self.conv_imag(x)
        # (batch_size, n_fft // 2 + 1, time_steps)

        real = real[:, None, :, :].transpose(2, 3)
        imag = imag[:, None, :, :].transpose(2, 3)
        # (batch_size, 1, time_steps, n_fft // 2 + 1)

        return real, imag
    
# 用于计算音频信号的频谱图    
class Spectrogram(nn.Module):
    def __init__(self, n_fft=2048, hop_length=512, win_length=1024, window='hann',
                 center=True, pad_mode='reflect', power=2.0, freeze_parameters=True):
        """Calculate spectrogram using pytorch. The STFT is implemented with 
        Conv1d. The function has the same output of librosa.core.stft
        """
        super(Spectrogram, self).__init__()
        self.power = power
        self.stft = STFT(n_fft=n_fft, hop_length=hop_length, win_length=win_length, 
                         window=window, center=center, pad_mode=pad_mode, freeze_parameters=True)

    # PyTorch中 forward 方法是在调用模型实例时自动触发的。当你调用一个继承自 nn.Module 的类的实例，
    # 并向其传递输入数据时，PyTorch 会自动调用该类中的 forward 方法来执行前向传播操作。
    def forward(self, input):
        """input: (batch_size, 1, time_steps, n_fft // 2 + 1)
        Returns:
          spectrogram: (batch_size, 1, time_steps, n_fft // 2 + 1)
        """
        (real, imag) = self.stft.forward(input)
        # (batch_size, n_fft // 2 + 1, time_steps)

        spectrogram = real ** 2 + imag ** 2

        if self.power == 2.0:
            pass
        else:
            spectrogram = spectrogram ** (self.power / 2.0)

        return spectrogram

class LogmelFilterBank(nn.Module):
    def __init__(self, sr=32000, n_fft=2048, n_mels=64, fmin=50, fmax=14000, is_log=True, ref=1.0,
                 amin=1e-10, top_db=80.0, freeze_parameters=True):
        """Calculate logmel spectrogram using pytorch. The mel filter bank is 
        the pytorch implementation of as librosa.filters.mel 
        """
        super(LogmelFilterBank, self).__init__()

        self.is_log = is_log
        self.ref = ref
        self.amin = amin
        self.top_db = top_db

        self.melW = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels,
            fmin=fmin, fmax=fmax).T
        # (n_fft // 2 + 1, mel_bins)

        self.melW = nn.Parameter(torch.Tensor(self.melW))

        if freeze_parameters:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, input):
        """input: (batch_size, channels, time_steps)
        
        Output: (batch_size, time_steps, mel_bins)
        """

        # Mel spectrogram
        mel_spectrogram = torch.matmul(input, self.melW)

        # Logmel spectrogram
        if self.is_log:
            output = self.power_to_db(mel_spectrogram)
        else:
            output = mel_spectrogram

        return output


    def power_to_db(self, input):
        """Power to db, this function is the pytorch implementation of 
        librosa.core.power_to_lb
        """
        ref_value = self.ref
        log_spec = 10.0 * torch.log10(torch.clamp(input, min=self.amin, max=np.inf))
        log_spec -= 10.0 * np.log10(np.maximum(self.amin, ref_value))

        if self.top_db is not None:
            if self.top_db < 0:
                raise ValueError('top_db must be non-negative')
            log_spec = torch.clamp(log_spec, min=log_spec.max().item() - self.top_db, max=np.inf)

        return log_spec

# 用于在输入的频谱图上随机丢弃条纹的模块
class DropStripes(nn.Module):
    def __init__(self, dim, drop_width, stripes_num):
        """Drop stripes. 
        Args:
          dim: int, dimension along which to drop
          drop_width: int, maximum width of stripes to drop
          stripes_num: int, how many stripes to drop
        """
        super(DropStripes, self).__init__()

        assert dim in [2, 1]    # dim 1:freq:指定的n_mels ; dim 2:time:算出来的

        self.dim = dim
        self.drop_width = drop_width
        self.stripes_num = stripes_num

    def forward(self, input):
        """input: (batch_size, channels, time_steps, freq_bins)"""
        assert input.ndimension() == 3
        
        if self.training is False:
            return input
        else:
            batch_size = input.shape[0]
            total_width = input.shape[self.dim]
            for n in range(batch_size):
                self.transform_slice(input[n], total_width)
            return input
        
    def transform_slice(self, e, total_width):
        """e: (channels, time_steps, freq_bins)"""
        for _ in range(self.stripes_num):
            distance = torch.randint(low=0, high=self.drop_width, size=(1,))[0]
            bgn = torch.randint(low=0, high=total_width - distance, size=(1,))[0]
            if self.dim == 1:
                e[bgn : bgn + distance, :] = 0
            elif self.dim == 2:
                e[:, bgn : bgn + distance] = 0

# 用于实现频谱图的数据增强操作，其中包含了时间维度和频率维度的条纹丢弃操作。
class SpecAugmentation(nn.Module):
    def __init__(self, time_drop_width, time_stripes_num, freq_drop_width, freq_stripes_num):
        """Spec augmetation. 
        [ref] Park, D.S., Chan, W., Zhang, Y., Chiu, C.C., Zoph, B., Cubuk, E.D. 
        and Le, Q.V., 2019. Specaugment: A simple data augmentation method 
        for automatic speech recognition. arXiv preprint arXiv:1904.08779.
        Args:
          time_drop_width: int
          time_stripes_num: int
          freq_drop_width: int
          freq_stripes_num: int
        """
        super(SpecAugmentation, self).__init__()
        
        self.freq_dropper = DropStripes(dim=1, drop_width=freq_drop_width,
                                        stripes_num=freq_stripes_num)
        
        self.time_dropper = DropStripes(dim=2, drop_width=time_drop_width,
                                        stripes_num=time_stripes_num)

    def forward(self, input):
        x = self.time_dropper(input)
        x = self.freq_dropper(x)
        return x

## 1st的模型函数

In [4]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)
    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)

def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.
    
    对输入的三维张量 x 在时间维度上进行插值，以补偿卷积神经网络 (CNN) 下采样时造成的分辨率降低。
    具体地，它通过重复时间步的方式进行插值。
    
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled

def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    
    将 framewise_output 张量填充到指定的帧数长度 frames_num，填充值为最后一帧的值。
    填充后的张量将与输入帧的长度一致。
    
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output

class ConvBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.conv2 = nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()

    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):

        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x

class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

### 构造数据集

In [22]:
class train_loader(Dataset):
    def __init__(self, train_list, num_frames, **kwargs):
        self.num_frames = num_frames
        # Load data & labels
        df_train = pd.read_csv(train_list)
        self.data_list = df_train['wav_path'].tolist()
        self.data_label = df_train['label'].tolist()
        self.torchfbank = torch.nn.Sequential(
            PreEmphasis(),            
            torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_fft=512, win_length=400, 
                                                 hop_length=160, window_fn=torch.hamming_window, 
                                                 n_mels=40))
        
        self.spectrogram_extractor = Spectrogram()
        self.logmel_extractor = LogmelFilterBank()
        self.spec_augmenter = SpecAugmentation(
            time_drop_width=64,
            time_stripes_num=2,
            freq_drop_width=8,
            freq_stripes_num=2)
        
    def process_audio_flag_1(self, feats):
        feats = torch.tensor(feats)  # [5, 16000]
        with torch.no_grad():
            processed_audio = self.torchfbank(feats)
            x = self.torchfbank(processed_audio)+1e-6
            x = x.log()  
            x = x - torch.mean(x, dim=-1, keepdim=True)
        return x.numpy()
    
    def process_audio_flag_2(self, feats):
        x = self.spectrogram_extractor(feats)  # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)  # (batch_size, 1, time_steps, mel_bins)
        x = self.spec_augmenter(x)
        return x.numpy()
        
    def __getitem__(self, index):
        # Read the utterance and randomly select the segment
        # 每次获取数据时随机选择一个片段进行训练，以增加数据多样性。（舍弃）
        
        audio, sr = soundfile.read(self.data_list[index])
        max_audio = self.num_frames * 80
        if audio.shape[0] <= max_audio:
            shortage = max_audio - audio.shape[0]
            audio = numpy.pad(audio, (0, shortage), 'wrap')
        feats = []
        startframe = numpy.linspace(0, audio.shape[0]-max_audio, num=5)
        for asf in startframe:
            feats.append(audio[int(asf):int(asf)+max_audio])
        feats = numpy.stack(feats, axis = 0).astype(np.float32) 
        print(feats.shape)  # (5, 16000)
        
        x1 = self.process_audio_flag_1(feats)
        x2 = self.process_audio_flag_2(feats)
        
        return  [x1, x2]

    def __len__(self):
        return len(self.data_list)

### 构造数据集2

In [11]:
class train_loader(Dataset):
    def __init__(self, train_list, num_frames, **kwargs):
        self.num_frames = num_frames
        # Load data & labels
        df_train = pd.read_csv(train_list)
        self.data_list = df_train['wav_path'].tolist()
        self.data_label = df_train['label'].tolist()
        
        self.spectrogram_extractor = Spectrogram()
        self.logmel_extractor = LogmelFilterBank()
        self.spec_augmenter = SpecAugmentation(
            time_drop_width=20,
            time_stripes_num=2,
            freq_drop_width=8,
            freq_stripes_num=2)
        
    
    def process_audio_flag_2(self, feats):
        feats = torch.tensor(feats)
        x = self.spectrogram_extractor(feats)  # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)  # (batch_size, 1, time_steps, mel_bins)
        x = self.spec_augmenter(x)
        return x.numpy()
    
        
    def __getitem__(self, index):
        # Read the utterance and randomly select the segment
        # 每次获取数据时随机选择一个片段进行训练，以增加数据多样性。（舍弃）
        
        audio, sr = soundfile.read(self.data_list[index])
        max_audio = self.num_frames * 80
        if audio.shape[0] <= max_audio:
            shortage = max_audio - audio.shape[0]
            audio = numpy.pad(audio, (0, shortage), 'wrap')
        feats = []
        startframe = numpy.linspace(0, audio.shape[0]-max_audio, num=5)
        for asf in startframe:
            feats.append(audio[int(asf):int(asf)+max_audio])
        feats = numpy.stack(feats, axis = 0).astype(np.float32) 
        x2 = self.process_audio_flag_2(feats)
        print(x2.shape)
        return  x2

    def __len__(self):
        return len(self.data_list)

In [12]:
filepath = "../user_data/train_data.csv"
trainloader = train_loader(filepath, num_frames=200)
trainLoader = torch.utils.data.DataLoader(trainloader, batch_size = 128, shuffle = True,
                                          num_workers = 0, drop_last = True)

In [13]:
for num, (data, labels) in enumerate(trainLoader, start = 1):
    if num ==1:
        print(data.shape)

(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)
(5, 1, 32, 64)


KeyboardInterrupt: 

# 定义模型

In [5]:
# 对音频信号进行预处理。预加重是一种常见的音频信号处理技术，
# 用于增强高频成分，以便在后续处理（如特征提取、分析等）中获得更好的性能。
class PreEmphasis(torch.nn.Module):
    def __init__(self, coef: float = 0.97):
        super().__init__()
        self.coef = coef
        self.register_buffer(
            'flipped_filter', torch.FloatTensor([-self.coef, 1.]).unsqueeze(0).unsqueeze(0))
        
    def forward(self, input: torch.tensor) -> torch.tensor:
        input = input.unsqueeze(1) # [128, 1, 16000]  # 这里仅加上bs
        input = F.pad(input, (1, 0), 'reflect')
        return F.conv1d(input, self.flipped_filter).squeeze(1)
    
class CNN(nn.Module):
    def __init__(self, in_features, nclass=2, **kwargs):
        super(CNN, self).__init__()
        
        self.torchfbank = torch.nn.Sequential(
            PreEmphasis(),            
            torchaudio.transforms.MelSpectrogram(sample_rate=32000, n_fft=1024, win_length=1024, 
                                                 hop_length=256, window_fn=torch.hamming_window, 
                                                 n_mels=64),  # [bs, n_mels, time_frames]
            # 这个里面就是放类的。
            SpecAugmentation(
                freq_drop_width=6, freq_stripes_num=2,
                time_drop_width=3, time_stripes_num=2,
                            )
        )
        
        self.fc1 = nn.Linear(2048, 1024, bias=True)
        self.att_block = AttBlock(1024, 1, activation='sigmoid')
        self.densenet_features = models.densenet121(pretrained=True).features
        
        self.resnet_features = models.resnet101(pretrained=True)
        self.resnet_features = nn.Sequential(*list(self.resnet_features.children())[:-2])  # Extract features only
        
        self.init_weight()
        
    def init_weight(self):
        init_layer(self.fc1)
        
    def cnn_feature_extractor(self, x):
        x = self.densenet_features(x)
        return x
    
    def resnet_feature_extractor(self, x):
        x = self.resnet_features(x)
        return x

    def forward(self, x: torch.Tensor):
        with torch.no_grad():
            x = self.torchfbank(x)+1e-6
            x = x.log()  
            x = x - torch.mean(x, dim=-1, keepdim=True)
        x = x.unsqueeze(1)  # torch.Size([bs, 1, n_mel(freq), time])
        x = x.expand(x.shape[0], 3, x.shape[2], x.shape[3]) # torch.Size([128, 3, 40, 101])
        x = self.resnet_feature_extractor(x)
        
        # Aggregate in frequency axis
        x = torch.mean(x, dim=3)
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2
        
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)

        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        return clipwise_output

In [9]:
class FakeModel(nn.Module):
    def __init__(self, lr, lr_decay, n_class, device, test_step, **kwargs):
        super(FakeModel, self).__init__()
        ## ResNet
        self.device = device
        self.speaker_encoder = CNN(in_features=1, nclass=n_class).to(self.device)
        
        ## 配置
        self.speaker_loss = nn.CrossEntropyLoss()
        # 定义二分类交叉熵损失函数
        self.speaker_loss = nn.BCELoss()
        
        self.optim = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = 2e-5)
        # 定义 AdamW 优化器
        self.optim = torch.optim.AdamW(self.parameters(), lr=lr, weight_decay=2e-5)  # weight_decay 表示权重衰减系数
        
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optim, 
                                                         step_size = test_step, gamma=lr_decay)
        print(time.strftime("%m-%d %H:%M:%S") + " Model para number = %.2f"%(sum(param.numel() for param in self.speaker_encoder.parameters()) / 1024 / 1024))

    def train_network(self, epoch, loader):
        self.train()
        ## Update the learning rate based on the current epcoh
        self.scheduler.step(epoch - 1)
        index, acc, loss, recall, F1 = 0, 0, 0, 0, 0
        lr = self.optim.param_groups[0]['lr']
        for num, (data, labels) in enumerate(loader, start = 1):
            self.zero_grad()
            labels = torch.LongTensor(labels).to(self.device) # （bs）
            
            outputs = self.speaker_encoder.forward(data.to(self.device))  # 二维（bs, 1）
            outputs = outputs.view(-1,)
            
            outputs = outputs.float()
            labels = labels.float()
            
            nloss = self.speaker_loss(outputs, labels)
            acc_t, recall_t, prec_t, F1_t = metrics_scores(outputs, labels)
            nloss.backward()
            self.optim.step()
            index += len(labels)
            acc += acc_t
            recall += recall_t
            F1 += F1_t
            loss += nloss.detach().cpu().numpy()
            sys.stderr.write(time.strftime("%m-%d %H:%M:%S") + \
            " [%2d] Lr: %5f, Training: %.2f%%, "%(epoch, lr, 100 * (num / loader.__len__())) + \
            " Loss: %.5f, Acc: %2.2f%%, Recall: %2.2f%%, F1: %2.2f%%\r" %(loss/(num), acc/index*len(labels), recall/index*len(labels), F1/index*len(labels)*100))
            sys.stderr.flush()
        sys.stdout.write("\n")
        return loss/num, lr, acc/index*len(labels), recall/index*len(labels), F1/index*len(labels)*100 

    def eval_network(self, eval_list, num_frames):
        self.eval()
        files = []
        outputs = torch.tensor([]).to(self.device)
        df_test = pd.read_csv(eval_list)
        label_list = list(np.array(df_test["label"].tolist()).reshape(-1, 1))
        
        setfiles = df_test["wav_path"].tolist()
        loss, top1 = 0, 0
        for idx, file in tqdm.tqdm(enumerate(setfiles), total = len(setfiles)):
            audio, _ = soundfile.read(file)

            # Spliited utterance matrix
            max_audio = num_frames * 80
            if audio.shape[0] <= max_audio:
                shortage = max_audio - audio.shape[0]
                audio = numpy.pad(audio, (0, shortage), 'wrap')
            feats = []
            startframe = numpy.linspace(0, audio.shape[0]-max_audio, num=5)
            for asf in startframe:
                feats.append(audio[int(asf):int(asf)+max_audio])
            feats = numpy.stack(feats, axis = 0).astype(np.float32)
            data_2 = torch.FloatTensor(feats).to(self.device)
            # Speaker embeddings
            with torch.no_grad():
                output = self.speaker_encoder.forward(data_2)
                output = torch.mean(output, dim=0).view(1, -1)
            outputs = torch.cat((outputs, output), 0)
        
        acc, recall, prec, F1 = metrics_scores(outputs, torch.tensor(label_list).to(self.device))
        return acc, recall, F1*100
    
    def save_parameters(self, path):
        torch.save(self.state_dict(), path)
        
    def load_parameters(self, path):
        self_state = self.state_dict()
        loaded_state = torch.load(path)
        for name, param in loaded_state.items():
            origname = name
            if name not in self_state:
                name = name.replace("module.", "")
                if name not in self_state:
                    print("%s is not in the model."%origname)
                    continue
            if self_state[name].size() != loaded_state[origname].size():
                print("Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state[origname].size()))
                continue
            self_state[name].copy_(param)

In [10]:
parser = argparse.ArgumentParser()
parser.add_argument('--device', type=str, default='cuda:0', help='Device training on ')
## Training Settings
parser.add_argument('--num_frames', type=int, default=500, help='Duration of the input segments, eg: 200 for 2 second')
parser.add_argument('--max_epoch', type=int, default=2000, help='Maximum number of epochs')
parser.add_argument('--batch_size', type=int, default=64, help='Batch size')

# jupyter中，dataloader的num_workers中能为0
parser.add_argument('--n_cpu', type=int, default=0, help='Number of loader threads')
parser.add_argument('--test_step', type=int, default=5, help='Test and save every [test_step] epochs')
parser.add_argument('--lr', type=float, default=0.001, help='Learning rate')
parser.add_argument("--lr_decay", type=float, default=0.97, help='Learning rate decay every [test_step] epochs')


parser.add_argument('--save_path',  type=str, default="../out_put/", help='Path to save the score.txt and models')
parser.add_argument('--initial_model',  type=str, default="", help='Path of the initial_model')

## Model and Loss settings
parser.add_argument('--n_class', type=int, default=2, help='Number of class')

## Command
parser.add_argument('--eval', dest='eval', action='store_true', help='Only do evaluation')

## Initialization
warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
args = parser.parse_args(args=[])
args = init_args(args)

In [46]:
# trainloader = train_loader(filepath, num_frames=args.num_frames)
# trainLoader = torch.utils.data.DataLoader(trainloader, batch_size = args.batch_size, shuffle = True,
#                                           num_workers = args.n_cpu, drop_last = True)

# model = CNN(in_features=1, nclass=1)
# for num, (data, labels) in enumerate(trainLoader, start = 1):
#     if num ==1:
#         print(labels.shape)
#         output = model(data)
#         print(output.shape)
#         outputs = output.view(-1,)
#         print(outputs.shape)
        
# output = output.detach().cpu().numpy()
# output = output > 0.5  # 这会创建一个布尔数组，True 表示原值大于0.5，False 表示小于等于0.5
# output = output.astype(int)  # 将布尔值转换为整数，True 变为 1，False 变为 0

torch.Size([64])
torch.Size([64, 1])
torch.Size([64])


In [11]:
epoch = 1
s = FakeModel(**vars(args))
warnings.simplefilter("ignore")
torch.multiprocessing.set_sharing_strategy('file_system')
filepath = "../user_data/train_data.csv"
trainloader = train_loader(filepath, num_frames=args.num_frames)
trainLoader = torch.utils.data.DataLoader(trainloader, batch_size = args.batch_size, shuffle = True,
                                          num_workers = args.n_cpu, drop_last = True)

ACCs = []
RECALLs = []
F1s = []
score_file = open(args.score_save_path, "a+")

while(1):
    print(epoch)
    ## Training for one epoch
    loss, lr, acc, recall, f1 = s.train_network(epoch = epoch, loader = trainLoader)

    ## Evaluation every [test_step] epochs
    if epoch % args.test_step == 0:
        s.save_parameters(args.model_save_path + "/model_%04d.model"%epoch)
        eval_list = "../user_data/eval_data.csv"
        acc_pred, recall_pred, f1_pred = s.eval_network(eval_list = eval_list,
                                                        num_frames=args.num_frames)
        ACCs.append(acc_pred)
        RECALLs.append(recall_pred)
        F1s.append(f1_pred)
        print(time.strftime("%Y-%m-%d %H:%M:%S"), "%d epoch, Acc %2.2f%%, Recall %2.2f%%, F1 %2.2f%%, Val_acc %2.2f%%, Best_acc %2.2f%%, Val_recall %2.2f%%, Best_recall %2.2f%%, Val_f1  %2.2f%%, Best_f1 %2.2f%%"%(epoch, acc, recall, f1, ACCs[-1], max(ACCs), RECALLs[-1], max(RECALLs), F1s[-1], max(F1s)))
        score_file.write("%d epoch, Acc %2.2f%%, Recall %2.2f%%, F1 %2.2f%%, Val_acc %2.2f%%, Best_acc %2.2f%%, Val_recall %2.2f%%, Best_recall %2.2f%%, Val_f1  %2.2f%%, Best_f1 %2.2f%%\n"%(epoch, acc, recall, f1, ACCs[-1], max(ACCs), RECALLs[-1], max(RECALLs), F1s[-1], max(F1s)))
        score_file.flush()

    if epoch >= args.max_epoch:
        quit()
        
    epoch += 1

05-25 01:42:54 Model para number = 49.17
1



2



3



4



5


...


1325


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 5100/5100 [01:29<00:00, 57.01it/s]


2024-05-26 12:00:03 1325 epoch, Acc 100.00%, Recall 99.99%, F1 99.99%, Val_acc 99.22%, Best_acc 99.33%, Val_recall 99.46%, Best_recall 99.79%, Val_f1  99.17%, Best_f1 99.29%
1326



1327


KeyboardInterrupt: 

# 测试

In [26]:
class Inferencer(object):
    def __init__(self, model_path):
        self.model = self.load_model(model_path)

    def load_model(self, model_path):
        model = CNN(in_features=1)
        self_state = model.state_dict()
        loaded_state = torch.load(model_path)
        for name, param in loaded_state.items():
            origname = name
            if name not in self_state:
                name = name.replace("speaker_encoder.", "")
                if name not in self_state:
                    print("%s is not in the model."%origname)
                    continue
            if self_state[name].size() != loaded_state[origname].size():
                print("Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state[origname].size()))
                continue
            self_state[name].copy_(param)

        model.load_state_dict(self_state)
        model.eval().cuda()
        return model

    def eval_embedding(self, speech_path, num_frames=500, max_audio=48000, channel='left'):
        #处理音频
        audio, _  = soundfile.read(speech_path)

        max_audio = num_frames*80
        if audio.shape[0] <= max_audio:
            shortage = max_audio - audio.shape[0]
            audio = np.pad(audio, (0, shortage), 'wrap')
        feats = []
        startframe = np.linspace(0, audio.shape[0]-max_audio, num=5)    
        for asf in startframe:
            feats.append(audio[int(asf):int(asf)+max_audio])

        feats = np.stack(feats, axis = 0).astype(np.float32)
        data = torch.FloatTensor(feats).cuda()
        
        #推断
        with torch.no_grad():
            outputs = self.model.forward(data)
            outputs = torch.mean(outputs, dim=0).view(1, -1)# 因为数据处理成5维的，所以要换成一维的
        output = outputs.detach().cpu().numpy()
        output = output > 0.5  # 这会创建一个布尔数组，True 表示原值大于0.5，False 表示小于等于0.5
        output = output.astype(int)  # 将布尔值转换为整数，True 变为 1，False 变为 0
        
        return output[0][0]
    
def main(speech_list, infer, res_path):
    result = []
    for idx, file in tqdm.tqdm(enumerate(speech_list), total = len(speech_list)):
        output = infer.eval_embedding(file)
        result.append([os.path.basename(file), output])
    df_result = pd.DataFrame(result, columns=["speech_name", "pred_label"])
    df_result.to_csv(res_path, index=False, header=None)
    return df_result
    

if __name__=="__main__":
    parser = argparse.ArgumentParser(description = "DeepFake audio")
    parser.add_argument('--model_path',      type=str,   default='../out_put/base_1st_res_model/model_0070.model',       help='Model checkpoint path')
    parser.add_argument('--test_path',      type=str,   default='../raw_data/finvcup9th_1st_ds5/test_data.csv',       help='Path of test file, strictly same with the original file')
    parser.add_argument('--save_path', type=str, default='../out_put/submit.csv', help='Path of result')
    args = parser.parse_args([])
    print('loading model...')
    infer = Inferencer(args.model_path)
    df_test = pd.read_csv(args.test_path)
   
    print('model inferring...')
    main(df_test["wav_path"].tolist(), infer, res_path=args.save_path)
    print('done!')

loading model...


F:\Anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
F:\Anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
F:\Anaconda\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the mos

model inferring...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 4894/4894 [03:09<00:00, 25.77it/s]

done!


### 后处理

In [1]:
import os, shutil
import librosa
import numpy as np
import glob
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [2]:
test_path = "../raw_data/finvcup9th_1st_ds5/test/"
sub = pd.read_csv("../out_put/submit.csv", header=None)
print(sub[1].value_counts())
test_dict = sub.set_index(0)[1].to_dict()
under_1s_test = {}
for i in sub[0]:
    filename = test_path + i
    y, sr = librosa.load(filename, sr=None)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration<1:
        under_1s_test[i] = 0
print(len(under_1s_test))

under_1s_testlabel = []
for i in under_1s_test:
    under_1s_testlabel.append(test_dict[i])
pd.Series(under_1s_testlabel).value_counts()

1
0    2471
1    2423
Name: count, dtype: int64


C:\Users\Huolj\AppData\Local\Temp\ipykernel_22604\4093830948.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(filename, sr=None)
F:\Anaconda\envs\pytorch\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/finvcup9th_1st_ds5/test/test\\00047facd39a8f1427ecc55e3809d370.wav'

In [14]:
test_path = "../raw_data/finvcup9th_1st_ds5/test/"
sub = pd.read_csv("../out_put/submit.csv")

In [ ]:
sub.iloc[:,0] = sub.iloc[:,0].str.replace(r'test\\', '')

In [22]:
sub.loc[sub[0].isin(list(under_1s_test.keys())), 1] = 1

In [23]:
sub[1].value_counts()

1
0    3237
1    1657
Name: count, dtype: int64

In [18]:
sub.to_csv("../out_put/submit_yhl.csv", header=0, index=0)